# Exercises for Week 11: Abstract Formulation

## Name: Yongbum Kim

## Grading Scheme:

- **3 (Essentially perfect)**: All required questions have a complete solution that is essentially correct. This means that the math formulation correctly models the given problem and the code outputs the correct results. However, it is okay if the code returns another optimal solution that is equally good. 
- **2 (Solid effort)**: Several required questions may be incomplete or there are serious errors or logical flaws, but at least two thirds has a correct solution.
- **1 (Some effort)**: Between one third and two thirds of the required questions have a correct solution.
- **0 (No submission or essentially blank)**: No submission on Brightspace by the deadline, or less than one third of the required questions have a correct solution.

Every question is required unless it is marked with "(optional)". **To ensure that you get 3 out of 3, before you submit, restart the Kernel and run all, and check the following:**

- All code outputs are exactly as the sample outputs given in the exercises PDF.  
- All math formulations have **linear** objectives and constraints, and models the given problem in a **correct** and **complete** way. This means that your constraints allow every solution that the problem text would allow, and disallow every solution that the problem text disallows. 

**You need to develop the habit of meticulously checking your outputs and math in order to ensure the best grade.**  The weekly exercises are intended to be completed in 4-5 hours, excluding class time. You should budget at least this much time per week for these exercises.

**After you submit, download the .ipynb file you uploaded to Brightspace and double check that you uploaded the correct file, and that every question has been properly saved! It is your responsibility to submit the correct file before the deadline.**

## Exercise 11.1: Abstract Formulation for Supply Chain Planning

Write the abstract formulation corresponding to the following Gurobi code from Week 10.

In [2]:
import pandas as pd
cost=pd.DataFrame([[20,18,21,8],[8,23,24,8],[25,8,8,19]],\
                 index=[1,2,3],columns=['A','B','C','D'])
demand=pd.Series([30,50,10,20],index=['A','B','C','D'])
capacity=pd.Series([40]*3, index=[1,2,3])
FCs=cost.index
regions=cost.columns
from gurobipy import Model, GRB
mod=Model()
x=mod.addVars(FCs,regions,name='x')
mod.setObjective(sum(cost.loc[f,r]*x[f,r] for f in FCs for r in regions))
for f in FCs:
    mod.addConstr(sum(x[f,r] for r in regions)<=capacity[f],name=f'Capacity_{f}')
for r in regions:
    mod.addConstr(sum(x[f,r] for f in FCs)>=demand[r],name=f'Demand_{r}')
mod.write('10-supplyChain.lp')
%cat 10-supplyChain.lp

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-15
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  20 x[1,A] + 18 x[1,B] + 21 x[1,C] + 8 x[1,D] + 8 x[2,A] + 23 x[2,B]
   + 24 x[2,C] + 8 x[2,D] + 25 x[3,A] + 8 x[3,B] + 8 x[3,C] + 19 x[3,D]
Subject To
 Capacity_1: x[1,A] + x[1,B] + x[1,C] + x[1,D] <= 40
 Capacity_2: x[2,A] + x[2,B] + x[2,C] + x[2,D] <= 40
 Capacity_3: x[3,A] + x[3,B] + x[3,C] + x[3,D] <= 40
 Demand_A: x[1,A] + x[2,A] + x[3,A] >= 30
 Demand_B: x[1,B] + x[2,B] + x[3,B] >= 50
 Demand_C: x[1,C] + x[2,C] + x[3,C] >= 10
 Demand_D: x[1,D] + x[2,D] + x[3,D] >= 20
Bounds
End


### Abstract Formulation

**Data:**
- $F$: set of FCs (fulfillment centers).
- $R$: set of regions.
- $c_{fr}$: unit cost of transporting from FC $f$ to region $r$.
- $q_{f}$: capacity of FC $f$.
- $d_{r}$: demand of region $r$.

**Decision Variables:**
- $x_{fr}$: amount to transport from FC $f$ to region $r$. (Continous)

**Objective:**
$$\text{Minimize: } \sum_{f \in F, r \in R} c_{fr}x_{fr} $$

**Constraints:**
$$\begin{aligned}
\text{(Capacity)} && \sum_{r \in R} x_{fr} & \le q_{f} && \text{for each FC $f \in F$.} \\
\text{(Demand)} && \sum_{f \in F} x_{fr} & \ge d_{r} && \text{for each region $r \in R$.} \\
\text{(Non-negativity)} && x_{fr} & \ge 0 && \text{for each FC $f \in F$ and each region $r \in R$.} \\
\end{aligned}$$

## Exercise 11.2: Abstract Formulation for Box Selection

Complete the abstract formulation corresponding to the following concrete formulation from Week 9.


| Item type | 1 | 2 | 3 | 
|--|--|--|--|
| Minimum box size (in cubit feet) | 1.5 | 1.7 | 2.0 |
| Demand | 400 | 500 | 200 |

### Concrete Formulation

**Decision Variables:**

- $Y_1, Y_2, Y_3$: how many boxes to make of each box type. (Integer)
- $Z_1, Z_2, Z_3$: whether to make the mold for each box type. (Binary)

**Objective and Constraints:**
$$\begin{aligned}
\text{Minimize:} && 1.5Y_1+1.7Y_2+2.0Y_3+&1000(Z_1+Z_2+Z_3) \\
\text{s.t.} \\
\text{(Demand 1)} && Y_1+Y_2+Y_3 & \ge 1100 \\
\text{(Demand 2)} && Y_2 + Y_3 & \ge 700 \\
\text{(Demand 3)} && Y_3 & \ge 200 \\
\text{(S boxes on/off)} && Y_1 & \le 1100Z_1 \\
\text{(M boxes on/off)} && Y_2 & \le 1100Z_2 \\
\text{(L boxes on/off)} && Y_3 & \le 1100Z_3 \\
&& Y_1,Y_2,Y_3& \ge 0
\end{aligned}$$

### Abstract Formulation

**Data:**
- $I$: Set of box type $i$.
- $s_{i}$: Minimum box size of box type $i$.
- $d_{i}$: Demand of box type $i$.
- $F$: Fixed cost of making the mold for each box type.

**Decision Variables:**

- $Y_{i}$: Number of boxes to make of each box type $i$. (Interger)
- $Z_{i}$: whether to make the mold for each box type $i$. (Binary)

**Objective and Constraints:**

$$\begin{aligned}
\text{Minimize} && \sum_{i=1}^n s_{i}Y_{i} + F\sum_{i=1}^n Z_{i} \\
\end{aligned}$$

$$\begin{aligned}
\text{s.t.} && \\
\text{(Demand)} && \sum_{j=i}^n Y_{j} & \ge \sum_{j=i}^n d_{j} && \text{for each $i \in I.$} \\
\text{(On/Off)} && Y_{i} & \le MZ_{i} && \text{for each $i \in I$.} \\
\text{(Non-negativity)} && Y_{i} & \ge 0 && \text{for each $i \in I$.} \\
\end{aligned}$$

## (Optional) Exercise 11.3: Optimal Debt Repayment

**In this question, you practice everything learnt in the second half of the course thus far, as you will write an English description, a concrete formulation, an abstract formulation, as well as Python code.**

Paris has come to you because she needs help paying off her credit card bills. Her statement at the beginning of month 1 shows the following balances:

| Credit Card | Balance | Monthly Rate |
|--|--|--|
| Saks Fifth Avenue | \$20,000 | 0.5\% |
| Bloomingdale's | \$50,000 | 1.0\% |
| Macy's | \$40,000 | 1.5\% |

Paris has agreed not to shop at any of these stores anymore, and she is willing to allocate up to 5,000 dollars per month to pay off these credit cards. All cards must be paid off within 36 months (meaning that her statement at the beginning of month 37 must be zero for all card). Paris’ goal is to minimize the total of all her payments.

For this problem, assume that the interest for the month is applied after the payment for that month. For example, suppose Paris pays 5,000 on Saks for month 1. Then her Saks balance at the beginning of month 2 is $(1.005)\times (20000 - 5000) = 15075$. 

Help Paris solve her problem by formulating it into a linear optimization model, then generalize it to be able to handle arbitrary number of credit cards, balances, monthly rates, monthly budget, and time required for full payment. 

### Step 1. English Description

Describe the decision, objective and constraints in English using precise, complete and succinct language.

**Decision:** How much money to pay for each credit card each month. (Continuous)

**Objective:** Minimize the total amount of payments within 36 months.

**Constraints:**
1) Monthly payment limit: $5,000 per month to pay off.
2) Full payment: All credit cards must be zero by the end of month 36.
3) Balance update: each credit card's balance for each month is updated on previous month's balance, payment and interest rate.
4) Non-negativity: each credit card's balance and payment for each month.

### Step 2. Concrete Formulation

Write a linear optimization formulation in which the only variables are decision variables, and all input data are represented as numbers.

**Decision variables:** 
- $x_{it}$: Amount of payment for credit card $i$ in month $t$. (Continuous)
- $y_{it}$: Balance of credit card $i$ at the end of month $t$. (Continuous)

**Objective and constraints:**
$$\begin{aligned}
\text{Minimize:} \qquad x_{S1} + x_{B1} + x_{M1} + x_{S2} + x_{B2} + x_{M2} + \cdots + x_{S36} + x_{B36} + x_{M36} \\
\end{aligned}$$

$$\begin{aligned}
\text{s.t.} \\
\text{(Monthly payment limit)} && x_{St} + x_{Bt} + x_{Mt} & \le 5000 && \text{for } t = 1,2,\cdots,36 \\
\text{(Full payment)} && y_{i36} & = 0 && \text{for } i = S,B,M \\
\text{(Initial Balance - S: month 0)} && y_{S0} & = 20000 \\
\text{(Initial Balance - B: month 0)} && y_{B0} & = 50000 \\
\text{(Initial Balance - M: month 0)} && y_{M0} & = 40000 \\
\text{(Balance update - S)} && y_{St} & = 1.005(y_{St-1} - x_{St}) && \text{for } t = 1,2,\cdots,36 \\
\text{(Balance update - B)} && y_{Bt} & = 1.010(y_{Bt-1} - x_{Bt}) && \text{for } t = 1,2,\cdots,36 \\
\text{(Balance update - M)} && y_{Mt} & = 1.015(y_{Mt-1} - x_{Mt}) && \text{for } t = 1,2,\cdots,36 \\
\text{(Non-negativity - Balance)} && y_{it} & \ge 0 && \text{for } i = S,B,M \text{ and for } t = 1,2,\cdots,36 \\
\text{(Non-negativity - Payment)} && x_{it} & \ge 0 && \text{for } i = S,B,M \text{ and for } t = 1,2,\cdots,36
\end{aligned}$$

### Step 3. Abstract Formulation

Generalize the concrete formulation to arbitrarily many credit cards, number of months, initial card balance, cash available in each month, and monthly interest rate. You may assume that the available cash is the same in each month, and that the interest rate for each card does not change over time.


**Data:**
- $I$: Set of credit cards.
- $n$: Number of months for full repayment
- $T$: Set of months. ($T = {1,\ldots,n}$)
- $c$: Monthly payment limit.
- $b_i$: Initial balance of credit card $i$.
- $r_i$: Monthly interest rate for credit card $i$.

**Decision variable:** 
- $x_{it}$: Amount paid to credit card $i$ in month $t$. (Continuous)
- $y_{it}$: Balance of credit card $i$ at the end of month $t$. (Continuous)


**Objective and constraints:** 
$$\begin{aligned}
\text{Minimize} \qquad &\sum_{i\in I,t\in T}x_{it} \\
\end{aligned}$$

$$\begin{aligned}
\text{s.t.} \\
\text{(Monthly payment limit)} && \sum_{i \in I}x_{it} & \le c && \text{for all $t \in T$.} \\
\text{(Full payment)} && y_{in} & = 0 && \text{for all $i \in I$.} \\
\text{(Initial balance)} && y_{i0} & = b_{i} && \text{for all $i \in I$.} \\
\text{(Balance update)} && y_{it} & = (1+r_i)(y_{it-1}-x_{it}) && \text{for all $i \in I$ and for all $t \in T$.} \\
\text{(Non-negativity)} && x_{it}, y_{it} & \ge 0 && \text{for all $i \in I$ and for all $t \in T$.} \\
\end{aligned}$$

### Step 4. Python Code

Implement your abstract formulation and find the numerical solution. The input data is given below.

You should print the minimum total payments and display a DataFrame listing the payments to each card each month. (See the expected output below.)

In [4]:
# Input data
I=['S','B','M']
n=36
T=range(1,n+1)
b={'S':20000,'B':50000,'M':40000}
c=5000
r={'S':0.005,'B':0.01,'M':0.015}

In [14]:
# Write your code here

from gurobipy import Model, GRB

mod = Model()
x = mod.addVars(I, T, vtype=GRB.CONTINUOUS, name='payment')
y = mod.addVars(I, [0]+list(T), vtype=GRB.CONTINUOUS, name='balance')

mod.setObjective(sum(x[i, t] for i in I for t in T), sense=GRB.MINIMIZE)

for t in T:
    mod.addConstr(sum(x[i, t] for i in I) <= c, name=f'monthly_limit_{t}')

for i in I:
    mod.addConstr(y[i, n] == 0, name='full_payment')
    mod.addConstr(y[i, 0] == b[i], name=f'initial_balance_{i}')
    for t in T:
        mod.addConstr(y[i, t] == (1 + r[i]) * (y[i, t-1] - x[i, t]), name=f'balance_update_{i}_{t}')

mod.setParam('OutputFlag', False)
mod.optimize()

print(f'Minimum total payments: {mod.objVal}')

Minimum total payments: 121797.50485862953


In [10]:
# Expected output

Minimum total payments: 121797.50485862953


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,482.259871,5000.0,5000.0,5000.0,5000.0,1797.504859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2736.955001,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,4517.740129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,2263.044999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Exercise 11.4: Assigning Consultants to Projects

Trojan Consulting would like to assign consultants to projects in a way that minimizes total travel costs while satisfying the skill requirements of each project and avoiding assigning the same consultant to two projects with overlapping time frames. 

In the following example, there are four consultants, each of whom may possess one or more of two possible skills. (A checkmark indicates whether the person has the skill.) Each project requires at least a certain number of consultants of each skill. If a consultant has both skills, he/she can count toward the number required for both skills, and the travel cost may potentially be less as one less person would be needed. Projects 1 and 2 have conflicting timelines, so the same consultant cannot be assigned to both. Similarly, projects 2 and 3 are also in conflict. But the same consultant may be assigned to projects 1 and 3.

|Consultant | Accounting | Operations |
|:--|--|--|
|Alice |√ |` `|
|Bob |√ |√|
|Charlie |` `| √ |
|Daphne |√ |√|

|Project| Accounting | Operations |
|--|--|--|
|P1 | 2 | 1 |
|P2 | 1 | 1 |
|P3 | 0 | 2 |

|Costs | P1 | P2 | P3 |
|--|--|--|--|
|Alice | 10 | 0 | 5 |
|Bob | 8 | 15 | 13 |
|Charlie | 0 | 5 | 10 |
|Daphne | 10 | 3 | 0 |

**Write an abstract formulation of a linear optimization model to find a cost-minimizing assignment that satisfies all of the above constraints.** Your formulation needs to be general enough to handle arbitrarily many consultants, projects, skills, as well as arbitrary information on skills of consultants, requirements of projects, conflicts between projects, and travel costs.

**Data:**
- $C$: Set of consultants.
- $P$: Set of projects.
- $S$: Set of skills.
- $K_{cp}$: Cost of assigning $c$ to $p$.
- $F$: Set of conflicting pairs of project. $F$ = {(1,2),(2,3)}
- $r_{ps}$: Number of people with skills $s$ needed in project $p$.
- $C_{s}$: Set of consultant $c$ with skill $s$.

**Decision Variables:**
- $x_{cp}$: whether to assign consultant $c$ to project $p$. (Binary)

**Objective and Constraints:**
$$\begin{aligned}
\text{Minimize} && \sum_{c \in C, p \in P} K_{cp}x_{cp} \\
\end{aligned}$$

$$\begin{aligned}
\text{s.t.} && \\
\text{(Conflicts)} && x_{cp1}+x_{cp2} & \le 1 && \text{for each $c \in C$ and for pair $(p_1, p_2) \in F.$}\\
\text{(Skill Requirement)} && \sum_{c \in C_{s}} x_{cp} & \ge r_{ps} && \text{for each $p \in P$ and $s \in S.$} \\
\end{aligned}$$

## Exercise 11.5: Classroom Seating under Social Distancing

A committee at USC Marshall is exploring the viability of in-person instruction while observing social distancing guidelines. One challenge is that certain classrooms have tables and seats bolted to the floor, and the seats cannot be moved unless the rooms undergo significant remodeling. As an illustration, the following image is a portion of the floor plan for JKP204, and the numbers in the image correspond to the individual seats. As you can see, the distance between adjacent seats can be quite close, and the room would be overly dense if every seat is used. Since the seats cannot be moved, only a subset of them can be used to seat students. 

![Illustration for 11.4](11-Seats.png)

**Your task is to formulate an optimization problem to maximize the number of students that can be safely seated in the current classrooms without remodeling.** The committee has access to the detailed floor plans of every classroom, and they have labelled every seat as above and measured its exactly position in terms of x-y coordinates, so they can easily compute the distance between any two seats. (For simplicity, the distance between two seated students is defined to be the straight-line distance between the center of the two seats.) Based on discussions with public health officials, the committee has summarized the requirements for safely seating students as follows:

1. The minimum distance between any two seated students is at least 6 feet.
2. For every seated student, the number of other students seated within a 9 feet radius is at most 3, so the density of the room is kept low. (In other words, if we draw a circle centered at a seated student with a radius of 9 feet, then there are at most 4 students seated strictly inside this circle, including the first student.)

**Write an abstract formulation of a LP/MIP to solve the above problem, by listing all the data variables, decision variables, objective, and constraints.** You may define any data variables that can be straightforwardly calculated based on the information the commitee has access to, but your definition must be completely clear and without ambiguities. Your formulation must be flexible enough to handle an arbitrary floor plan, not only the one shown above, and your objective and constraints must all be linear. 

**Data:**

- $S$: Set of seats.
- $C$: Set of pairs of seats that are within 6-feet.
- $N_{s}$: Set of other seats within 9-feet of seat $s$.
- $M$: the total number of seats in the classroom.

**Decision Variables:**
- $x_{s}$: whether to use each seat $s$. (Binary)

**Objective and Constraints:**

$$\begin{aligned}
\text{Maximize} && \sum_{s \in S} x_{s} \\
\end{aligned}$$

$$\begin{aligned}
\text{s.t.} && \\
\text{(Minimum Distance)} && x_{s}+x_{t} & \le 1 && \text{for each $(s, t) \in C.$}\\
\text{(Density - Conditional RHS)} && \sum_{t \in N_s} x_{t} & \le 3x_{s} + M(1-x_{s}) && \text{for each $s \in S$}
\end{aligned}$$

## Exercise 11.6: Supply Chain Planning Revisited

This question modifies the supply chain planning problem of Exercise 8.2 by for both production and transportation. You company produces and sells a certain product. There is a certain set of production plants, each with a given capacity, which is the maximum number of units that can be produced there in a given month. There is a certain set of demand regions, each with a given estimated demand, which is the maximum number of units you can sell in that region per month. It is possible to not fulfill all the demand, as that corresponds to having the item being stocked out for some customers. The price that you charge when sell the item may be different in each region, but the prices are determined by another department, so is outside of your control. 

As an illustration, suppose there are 3 plants, with the following production costs and capacities:

|Plant | 1 | 2 | 3 |
|--|--|--|--|
|Cost per unit | 60 | 60 | 55 |
|Capacity | 35 | 40 | 35 |

Suppose there are 4 regions, with the following selling prices and demand estimates:

| Region | A | B | C | D|
|--|--|--|--|--|
|Price  |80 | 85 | 70 | 75 |
|Demand |30 | 50 | 10 | 20 |


The following table provides the transportation cost of shipping each unit from each plant to each region.

| Region \ Plant  | 1 | 2 | 3 |
|--|--|--|--|
| A | 20 | 8 | 25 | 
| B| 18 | 23 | 8 | 
| C| 21 | 24 | 8 | 
| D | 8 | 8 | 19 |

Create an abstract formulation of a linear optimization problem to determine a plan for producing and shipping items, so as to maximize total profit, which is the revenue from the selling the product minus the production and transportation costs. You formulation must be flexible enough to handle arbitrary sets of production plants and regions, and none of the numbers in the above description can be hardcoded.

(In this problem, fractional units are allowed, since we are working with rates.)

**Data:**
- $P$: Set of plants.
- $R$: Set of regions.
- $c_{p}$: Production cost per each unit of plant $p$.
- $k_{p}$: Capacity of plant $p$.
- $p_{r}$: Selling price per unit of region $r$.
- $d_{r}$: Demand estimates of region $r$.
- $t_{pr}$: Transportation cost per unit from plant $p$ to region $r$.

**Decision Variables:**
- $x_{pr}$: Number of units produced at plant $p$ and shipped to region $r$. (Continous)
- $y_{r}$: Number of units sold in region $r$. (Continous)

**Objective:**
$$\begin{aligned}
\text{Maximize} && \sum_{r \in R} p_{r} y_{r} - \sum_{p \in P, r \in R} (c_{p} + t_{pr}) x_{pr} \\
\end{aligned}$$

**Constraints:**

$$\begin{aligned}
\text{(Capacity)} && \sum_{r \in R} x_{pr} & \le k_{p} && \text{for each $p \in P$.} \\
\text{(Production - Selling)} && \sum_{p \in P} x_{pr} & \ge y_r && \text{for each $r \in R$.} \\
\text{(Selling - Demand)} && y_r & \le d_{r} && \text{for each $r \in R$.} \\
\text{(Non-negativity)} && x_{pr}, y_{r} & \ge 0 && \text{for each $p \in P$ and for each $r \in R$.} \\
\end{aligned}$$